# Loading Libraries and helper files

In [1405]:
import os
from pathlib import Path
import sys
import warnings

import numpy as np
import pandas as pd
from mizani.formatters import percent_format
from plotnine import *
from scipy.stats import logistic
from scipy.stats import norm
from stargazer.stargazer import Stargazer
from patsy import dmatrices
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import log_loss
from plotnine import *

from stargazer import stargazer
from statsmodels.tools.eval_measures import mse,rmse

warnings.filterwarnings("ignore")

In [1406]:
path = Path(os.getcwd())
tech_prep = os.path.join(str(path), "utils/")
sys.path.append(tech_prep)
from py_helper_functions import *

# 1. Data Loading, Exploration, and Basic Filtering

In [1407]:
# Loading data and checking
main_df = pd.read_csv('https://osf.io/4ay9x/download')

In [1408]:
main_df.shape

(149316, 23)

### Filtering for my chosen occupation: Eduaction Occupations

In [1409]:
df = main_df.loc[main_df['occ2012'].isin([2200, 2300, 2310, 2320, 2330, 2340, 2400, 2430, 2440, 2540, 2550])]
df.shape

(9981, 23)

In [1410]:
# Checking missing values to see if I should correct/fill the dataset

missing_values = df.isnull().sum()

print("Missing values in each column:")
print(missing_values)

# No missing values in important/chosen variables

Missing values in each column:
Unnamed: 0       0
hhid             0
intmonth         0
stfips           0
weight           0
earnwke          0
uhours           0
grade92          0
race             0
ethnic        9202
age              0
sex              0
marital          0
ownchild         0
chldpres         0
prcitshp         0
state            0
ind02            0
occ2012          0
class            0
unionmme         0
unioncov      3733
lfsr94           0
dtype: int64


In [1411]:
df.describe()

,Unnamed: 0,hhid,weight,earnwke,uhours,grade92,race,ethnic,age,sex,marital,ownchild,chldpres,occ2012
count,9981.000000,9.981000e+03,9981.000000,9981.000000,9981.000000,9981.000000,9981.000000,779.000000,9981.000000,9981.000000,9981.000000,9981.000000,9981.000000,9981.000000
mean,156836.258992,4.451023e+14,2263.007325,935.652575,38.321811,42.961928,1.345456,2.658537,42.000902,1.755335,2.866847,0.780683,2.109708,2333.405470
std,92829.912325,3.217686e+14,1278.307226,585.607374,10.892425,1.816246,1.290666,2.437172,12.002275,0.429910,2.595665,1.081402,3.227588,95.613067
min,15.000000,8.171510e+09,53.232500,0.230000,1.000000,31.000000,1.000000,1.000000,16.000000,1.000000,1.000000,0.000000,0.000000,2200.000000
25%,76232.000000,1.333500e+14,1144.527300,500.000000,36.000000,43.000000,1.000000,1.000000,32.000000,2.000000,1.000000,0.000000,0.000000,2310.000000
50%,154275.000000,4.089016e+14,2528.657200,865.000000,40.000000,43.000000,1.000000,1.000000,42.000000,2.000000,1.000000,0.000000,0.000000,2310.000000
75%,238782.000000,7.250320e+14,3239.709400,1222.220000,40.000000,44.000000,1.000000,4.000000,52.000000,2.000000,5.000000,2.000000,3.000000,2330.000000
max,317003.000000,9.999605e+14,24595.832400,2884.610000,99.000000,46.000000,23.000000,8.000000,64.000000,2.000000,7.000000,9.000000,15.000000,2550.000000


In [1412]:
# Creating our target variable earnings per hour 'eph' and log of the same 'lneph'

df['eph'] = df['earnwke'] / df['uhours']
df['lneph'] = np.log(df['uhours'])

df['eph'].describe()

count    9981.000000
mean       24.406499
std        19.775815
min         0.004107
25%        14.423000
50%        21.250000
75%        30.000000
max      1076.920000
Name: eph, dtype: float64

#### As you can see the minimum  is 0.004, so there is an error in the data as for a 40 hour work week the pay would be 0.16, which is impossible. I will be removing values less than 7.5 Dollars as that was the minimum hourly wage in 2013 for US

In [1413]:
# Basic filtering for working age and cleaner data 

df = df.loc[(df['age'] >= 18) # Filtering for at least 18 years of age
             & (df['earnwke'] > 0) # Filtering for more than 0 wage
             & (df['eph'] > 7.5)
                ]                     
df.shape

(9600, 25)

## 2. Exploring chosen predictor variables
    1. Education level
    2. Age
    3. Gender
    4. Married
    5. has children
    6. Union
    7. Employment type
    8. Private or Public
    9. Race

In [1414]:
# Checking what our school teachers education distribution

freq = df.groupby("grade92").agg(frequency=("grade92", "size"))
freq["percent"] = round(freq["frequency"] / sum(freq["frequency"]) * 100, 3)
freq["cumulative_percent"] = np.cumsum(freq["percent"])
freq

,frequency,percent,cumulative_percent
grade92,,,
31,2,0.021,0.021
32,1,0.010,0.031
33,2,0.021,0.052
34,6,0.062,0.114
35,7,0.073,0.187
36,6,0.062,0.249
37,15,0.156,0.405
38,11,0.115,0.520
39,585,6.094,6.614


In [1415]:
# Checking what our school teachers gender distribution

freq = df.groupby("sex").agg(frequency=("sex", "size"))
freq["percent"] = round(freq["frequency"] / sum(freq["frequency"]) * 100, 3)
freq["cumulative_percent"] = np.cumsum(freq["percent"])
freq

,frequency,percent,cumulative_percent
sex,,,
1,2371,24.698,24.698
2,7229,75.302,100.000


In [1416]:
# Checking what our school teachers marital status distribution

freq = df.groupby("marital").agg(frequency=("marital", "size"))
freq["percent"] = round(freq["frequency"] / sum(freq["frequency"]) * 100, 3)
freq["cumulative_percent"] = np.cumsum(freq["percent"])
freq

,frequency,percent,cumulative_percent
marital,,,
1,6218,64.771,64.771
2,48,0.500,65.271
3,92,0.958,66.229
4,115,1.198,67.427
5,847,8.823,76.250
6,139,1.448,77.698
7,2141,22.302,100.000


In [1417]:
# Checking what our school teachers dependants(children) distribution

freq = df.groupby("ownchild").agg(frequency=("ownchild", "size"))
freq["percent"] = round(freq["frequency"] / sum(freq["frequency"]) * 100, 3)
freq["cumulative_percent"] = np.cumsum(freq["percent"])
freq

,frequency,percent,cumulative_percent
ownchild,,,
0,5519,57.490,57.490
1,1637,17.052,74.542
2,1678,17.479,92.021
3,587,6.115,98.136
4,138,1.438,99.574
5,29,0.302,99.876
6,7,0.073,99.949
7,4,0.042,99.991
9,1,0.010,100.001


In [1418]:
# Checking what our school teachers union membership distribution

freq = df.groupby("unionmme").agg(frequency=("unionmme", "size"))
freq["percent"] = round(freq["frequency"] / sum(freq["frequency"]) * 100, 3)
freq["cumulative_percent"] = np.cumsum(freq["percent"])
freq

,frequency,percent,cumulative_percent
unionmme,,,
No,5921,61.677,61.677
Yes,3679,38.323,100.000


In [1419]:
# Checking what our school teachers employment distribution

freq = df.groupby("lfsr94").agg(frequency=("lfsr94", "size"))
freq["percent"] = round(freq["frequency"] / sum(freq["frequency"]) * 100, 3)
freq["cumulative_percent"] = np.cumsum(freq["percent"])
freq

,frequency,percent,cumulative_percent
lfsr94,,,
Employed-Absent,850,8.854,8.854
Employed-At Work,8750,91.146,100.000


In [1420]:
# Checking what our school teachers sector distribution

freq = df.groupby("class").agg(frequency=("class", "size"))
freq["percent"] = round(freq["frequency"] / sum(freq["frequency"]) * 100, 3)
freq["cumulative_percent"] = np.cumsum(freq["percent"])
freq

,frequency,percent,cumulative_percent
class,,,
Government - Federal,134,1.396,1.396
Government - Local,4291,44.698,46.094
Government - State,2092,21.792,67.886
"Private, For Profit",1912,19.917,87.803
"Private, Nonprofit",1171,12.198,100.001


In [1421]:
# Checking what our school teachers sector distribution

freq = df.groupby("race").agg(frequency=("race", "size"))
freq["percent"] = round(freq["frequency"] / sum(freq["frequency"]) * 100, 3)
freq["cumulative_percent"] = np.cumsum(freq["percent"])
freq

,frequency,percent,cumulative_percent
race,,,
1,8206,85.479,85.479
2,786,8.188,93.667
3,56,0.583,94.250
4,400,4.167,98.417
5,31,0.323,98.740
6,22,0.229,98.969
7,33,0.344,99.313
8,20,0.208,99.521
9,7,0.073,99.594


_________________________________

## 3. Feature Engineering

Creating binary variables from categorical values from our chosen variables

In [1422]:
# Creating Education variables
df['HS_GED'] = (df['grade92'] == 39).astype(int) # Associate degree (Vocational/occupational)
df['college_unfin'] = (df['grade92'] == 40).astype(int) # Associate degree (Vocational/occupational)
df['AD_V'] = (df['grade92'] == 41).astype(int) # Associate degree (Vocational/occupational)
df['AD_AP'] = (df['grade92'] == 42).astype(int) # Associate degree (Academic Program)
df['BD'] = (df['grade92'] == 43).astype(int) # Bachelor's degree (e.g.BA,AB,BS)
df['MD'] = (df['grade92'] == 44).astype(int) # Master's degree (e.g.MA,MS,MEng,Med,MSW,MBA) 
df['PD'] = (df['grade92'] == 45).astype(int) # Professional degree (e.g.MD,DDS,DVM,LLB,JD)
df['PhD'] = (df['grade92'] == 46).astype(int) # Doctorate degree(e.g.PhD,EdD)



# Creating age squared variable
df["agesq"] = df["age"] ** 2




# Creating female variable
df['female'] = np.where(df['sex'] == 2, '1', '0') .astype(int)

In [1423]:
# Creating dummy variables for marital status
# Created 3 variables by grouping each categorical value that made sense
df['married'] = np.where(df['marital'] <=2, '1', '0').astype(int)
df['separated'] = np.where((df['marital'] >= 3) & (df['marital'] <= 6), '1', '0').astype(int)
df['never_married'] = np.where(df['marital'] == 7, '1', '0').astype(int)



# Creating dummy variables for employment category
df['emp_work']   = (df['lfsr94'] == "Employed-At Work").astype(int).astype(int)
df['emp_absent'] = (df['lfsr94'] == "Employed-Absent").astype(int).astype(int)




# Creating dummy variable for union membership
# Created 1 variable for Yes (1) or No (0)
df['is_union_member'] = np.where(df['unionmme'] == 'Yes', '1', '0').astype(int)

In [1424]:
# Creating dummy variables for class/sector of employment

df['gov_fed_empl']      = np.where(df['class'] == 'Government - Federal', '1', '0').astype(int)
df['gov_local_empl']    = np.where(df['class'] == 'Government - Local', '1', '0').astype(int)
df['gov_state_empl']    = np.where(df['class'] == 'Government - State', '1', '0').astype(int)
df['priv_prof_empl']    = np.where(df['class'] == 'Private, For Profit', '1', '0').astype(int)
df['priv_nonprof_empl'] = np.where(df['class'] == 'Private, Nonprofit', '1', '0').astype(int)



# Creating dummy variables for race
df['white_person'] = np.where(df['race'] == 1, '1', '0').astype(int)
df['non_white_person'] = np.where(df['race'] > 1 , '1', '0').astype(int)



# Creating dummy variables for citizenship
df['us_native'] = np.where(df['prcitshp'] == 'Native, Born In US', '1', '0').astype(int)
df['us_native_parents'] = np.where(df['prcitshp'] == 'Native, Born Abroad Of US Parent(s)', '1', '0').astype(int)
df['us_native_terr'] = np.where(df['prcitshp'] == 'Native, Born in PR or US Outlying Area', '1', '0').astype(int)
df['foreign_us'] = np.where(df['prcitshp'] == 'Foreign Born, US Cit By Naturalization', '1', '0').astype(int)
df['foreign_nonus'] = np.where(df['prcitshp'] == 'Foreign Born, Not a US Citizen', '1', '0').astype(int)


## Descriptive statistics

In [1425]:
summary_i = df[['eph','age','female',"HS_GED","college_unfin","AD_V", "AD_AP", "BD", "MD", "PD", "PhD",
                "married","separated","never_married", "ownchild", "is_union_member","emp_work","emp_absent",
                "gov_fed_empl", "gov_local_empl" , "gov_state_empl","priv_prof_empl","priv_nonprof_empl",
                "white_person","non_white_person",
                "us_native", "us_native_parents","us_native_terr","foreign_us","foreign_nonus"]].describe().round(3).T


summary = summary_i.rename(
    columns={
        "count": "Observations",
        "mean": "Mean",
        "std": "Standard Deviation",
        "25%": "25th Percentile",
        "50%": "Median",
        "75%": "75th Percentile",
        "max": "Maximum",
        "min": "Minimum",
    })
summary

,Observations,Mean,Standard Deviation,Minimum,25th Percentile,Median,75th Percentile,Maximum
eph,9600.0,25.134,19.812,7.517,15.0,21.634,30.288,1076.92
age,9600.0,42.232,11.833,18.000,32.0,42.000,52.000,64.00
female,9600.0,0.753,0.431,0.000,1.0,1.000,1.000,1.00
HS_GED,9600.0,0.061,0.239,0.000,0.0,0.000,0.000,1.00
college_unfin,9600.0,0.071,0.256,0.000,0.0,0.000,0.000,1.00
AD_V,9600.0,0.017,0.130,0.000,0.0,0.000,0.000,1.00
AD_AP,9600.0,0.040,0.196,0.000,0.0,0.000,0.000,1.00
BD,9600.0,0.350,0.477,0.000,0.0,0.000,1.000,1.00
MD,9600.0,0.373,0.484,0.000,0.0,0.000,1.000,1.00
PD,9600.0,0.017,0.131,0.000,0.0,0.000,0.000,1.00
